In [ ]:
# library pandas digunakan untuk memngekstrak dataframe dan mengubah spreadsheet menjadi format yang mudah diolah komputer
import pandas as pd
# numpy digunakan untuk menghitung sum dari semua angka
import numpy as np
# os digunaka untuk mengambil nama-nama file yang akan digunakan
# re digunakan untuk mengekstrak tahun dari nama file
import os, re

In [ ]:
# pada bagian ini file yang akan dipreprocess dipilih
files = [i for i in os.listdir('.') if i.endswith('xlsx')]

In [ ]:
# fungsi ini digunakan untuk mengubah suatu row menjadi nama column
"""
df adalah dataframe yang akan digunakan ditransformasi
row adalah lokasi row yang akan menjadi nama column
fillna adalah text yang akan digunakan untuk mengisi na pada column yang akan dibersihkan
"""
def replaceColumnLabel(df, row = 0, fillna = ''):
  col = df.iloc[row].fillna(fillna)
  df = df.iloc[row+1:]
  df.columns = col
  return df

In [ ]:
"""
process adalah function yang akan digunakan untuk mengubah data menjadi format yang lebih mudah dibaca
"""
def process(path):
  # pada line ini tahun dalam judul file akan diekstrak
  year = re.findall('\d+', path)[0]

  # kita membuat sebuah dataframe dari sebuah excel, pada file excel terdapat header sehingga perlu di skip row
  df = pd.read_excel(path, skiprows=1)


  # pada bagian ini data international dan national diprocess
  # pada kita extract semua data penerbangan international
  international = df.loc[:,'Keberangkatan pada Penerbangan Internasional':]
  # kita ubah nama column dan hilangkan semua row na
  international = replaceColumnLabel(international, row=1, fillna='airport').dropna()
  # dataframe kita ubah menjadi array 1 dimensi agar sum dari semua penerbangan dapat didapatkan
  international = np.ravel(international)
  # delete semua value string berupa '-'
  international = np.delete(international, np.where(international == '-'))
  # ubah tipe data menjadi int (hal ini dilakukan karena array memiliki bentuk data object) dan lakukan sum
  international_sum = np.sum(international.astype(int))

  # pada kita extract semua data penerbangan national
  national = df.loc[:, 'Keberangkatan pada Penerbangan Domestik':'Keberangkatan pada Penerbangan Internasional'].drop('Keberangkatan pada Penerbangan Internasional', axis=1)
  # kita ubah nama column dan hilangkan semua row na
  national = replaceColumnLabel(national, row=1, fillna='airport').dropna()
  # dataframe kita ubah menjadi array 1 dimensi agar sum dari semua penerbangan dapat didapatkan
  national = np.ravel(national)
  # delete semua value string berupa '-'
  national = np.delete(national, np.where(national == '-'))
  # ubah tipe data menjadi int (hal ini dilakukan karena array memiliki bentuk data object) dan lakukan sum
  national_sum = np.sum(national.astype(int))

  # gabungkan total sum international dan national dan beri label nasional maupun internasional
  # data diubah menjadi dataframe agar mudah digabungkan dengan data dari spreadsheet lain
  res_df = pd.DataFrame({'tahun': [year, year], 'Jumlah': [national_sum, international_sum], 'penerbangan':['nasional', 'internasional']})
  
  # kembalikan hasil dari preprocess
  return res_df

In [ ]:
# process semua file spreadsheet dan masukkan ke dalam sebuah list
dfs = [process(i) for i in files]

In [ ]:
# concatenate semua dataframe yang telah diproses menjadi 1 dataframe
df = pd.concat(dfs).sort_values('tahun')

# print dataframe untuk memastikan format sudah benar
df

,tahun,Jumlah,penerbangan
0,2019,36494088,nasional
1,2019,17108874,internasional
0,2020,32511290,nasional
1,2020,6520358,internasional
0,2021,8759617,nasional
1,2021,356016,internasional


In [ ]:
# simpan data ke dalam bentuk csv, index false agar csv tidak menyimpan index dari dataframe tersebut
df.to_csv('total keberangkatan melalui airport 2019-2020.csv', index=False)